In [ ]:
using Plots
using Random
include("src/fov.jl")
include("src/rrt_star.jl")
include("src/world.jl")
include("src/visual_odometry.jl")
include("src/nominal_planner.jl")
include("src/backup_planner.jl")
include("src/utils.jl")
include("src/gatekeeper.jl")
include("src/plotting.jl")


In [ ]:
using PythonCall
cv2 = pyimport("cv2")
image = cv2.imread("images/field.png")
gray_im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
features = cv2.goodFeaturesToTrack(gray_im, 1000, 0.04, 10)
cornersPlot = dropdims(pyconvert(Array{Int}, features), dims=2)
domain_size = 200
h,w = gray_im.shape
AR = pyconvert(Float64, w)/pyconvert(Float64, h)
# convert pixel coordinates to meters 
corners = cornersPlot .* domain_size/pyconvert(Float64,h)
#flip y axis
corners[:,2] = domain_size .- corners[:,2]
@time features = BallTree(corners', Euclidean(), reorder = false)

# if this ever fails, delete the .CondaPkg folder and run this section again

In [ ]:
Random.seed!(1337)
using StaticArrays
using .World
domain = (@SVector[0.0, 0.0, 0.0], @SVector[domain_size*AR, domain_size, 2*pi])
goal = @SVector [230.0, 175.0, 1.2*pi/2] # x, y, yaw

# can add unsafe zones here
# zone1 = World.Circle(@SVector[95.0, 125.0],30.0)
# zone4 = World.Circle(@SVector[100.0, 100.0],50.0)
# zone5 = World.Circle(@SVector[40.0, 110.0],20.0)
# zone2 = World.Rectangle(@SVector[95.0, 135.0], 80.0, 30.0, -2pi/5)
# zone3 = World.Rectangle(@SVector[80.0, 30.0], 40.0, 10.0, pi/4)
unsafe_zones = World.AbstractUnsafeZone[]

@time nominal_planner = initialize_nominal_planner(goal, domain, unsafe_zones, 10000)


In [ ]:
Random.seed!(1337)
using .VO
vo_params = VO.VOParams(fovRadius = 60.0, fovAngle = 90*pi/180, errorRate = 0.03, updateRate = 5.0, minFeatures = 8)
start_yaw = -pi/2
x0 = SVector(30.0,130.0,start_yaw)
starting_features_idx = features_in_fov_idx(features, x0, vo_params.fovRadius, 2pi)
seen_features = BallTree(features.data[starting_features_idx], Euclidean(), reorder = false)
mid_landmarks = [
    @SVector[165.0, 50.0, 0.0], 
]
landmarks = [x0, mid_landmarks..., goal]

backup_planner = 
    initialize_backup_planner(domain,
        vo_params,
        landmarks,
        MVector(x0[1], x0[2], vo_params.fovRadius),
        SizedVector(seen_features),
        unsafe_zones,
        100)

_, initial_nom_path = query_nominal_planner(nominal_planner,x0)
initial_nom_path = make_path_from_waypoints(initial_nom_path, nominal_planner.prob.turning_radius)

initial_traj = CompositeTrajectory([],[],0.0,0.0,0.0)
max_cost = 9.0
@time new_committed, committed_traj = gatekeeper(x0, initial_traj, nominal_planner, backup_planner, max_cost)
if new_committed
    println("first gk finished w/ cost: ", committed_traj.total_cost, " and switch_time: ", committed_traj.switch_time)
else
    println("first gk failed - must have a committed trajectory to start with")
end



In [ ]:
ΔT = 1/backup_planner.prob.vo_params.updateRate
current_cost = 0.0

x_hist = [x0]
gk_hist = [committed_traj]
features_hist = [seen_features]
backup_hist = [copy(backup_planner.nodes)]
# nom_path_hist = [nom_path]
cost_hist = [current_cost]
num_feats_hist = [NaN]
new_committed_hist = [1.0]
x = x0
t = t_commit = 0.0

mid_landmark_seen = false

@time begin
   for i = 1:800
      t += ΔT
      t_backup_reached = t_commit + committed_traj.switch_time + committed_traj.backup_time
      x = sample_committed_trajectory(committed_traj, t - t_commit, nominal_planner.prob.max_velocity)
      
      push!(x_hist, x)

      landmark_seen = false
      for l in backup_planner.prob.landmarks
         if t >= t_backup_reached || is_in_fov(x, l, backup_planner.prob.turning_radius*1.1, 2pi)#|| is_in_fov(x, l, backup_planner.prob.vo_params.fovRadius, backup_planner.prob.vo_params.fovAngle)
            i % 10 == 0 && println("Iter $i: landmark seen")
            current_cost = 0.0
            landmark_seen = true
            break
         end
      end
      
      if !landmark_seen
         cost, num_feats = VO.odometry_error(x_hist[end-1], x, features, vo_params)
         current_cost += cost
      else
         num_feats = NaN
      end
      push!(num_feats_hist, num_feats)
      push!(cost_hist, current_cost)

      i % 10 == 0 &&println("Iter $i: x: $x, cost: $current_cost")
      goal_reached = false
      if norm(x[SOneTo(2)] - backup_planner.prob.landmarks[end]) <= backup_planner.prob.turning_radius
         println("Iter $i: goal reached")
         goal_reached = true
      end

      new_features_idx = features_in_fov_idx(features, x, vo_params.fovRadius, vo_params.fovAngle)
      if i % 5 == 0 && length(backup_planner.nodes) < 4000
         update_backup_planner!(backup_planner,
            MVector(x[1], x[2], vo_params.fovRadius * 2),
            Set(features.data[new_features_idx]),
            10)
      end
      push!(features_hist, backup_planner.prob.mapped_features[1])
      push!(backup_hist, copy(backup_planner.nodes))

      new_committed, committed_traj = gatekeeper(x, committed_traj, nominal_planner, backup_planner, max_cost - current_cost)
   
      if new_committed
         i % 10 == 0 &&println("Iter $i: gk finished w/ cost: ", committed_traj.total_cost, "and switch_time: ", committed_traj.switch_time)
         t_commit = t
      else
         i % 10 == 0 &&println("Iter $i: gk failed")
      end
      push!(gk_hist, committed_traj)
      push!(new_committed_hist, new_committed ? 1 : NaN)

      if current_cost > max_cost
         println("Iter $i: max cost reached, cost = $current_cost")
         break
      end
      if goal_reached
         break
      end
   end
end

t_f = t

In [ ]:
function find_destination_index(nodes::Vector{RRTStar.Node{TV}}, start::Int64) where TV
    if nodes[start].parent_index == 0
        return start
    else
        return find_destination_index(nodes, nodes[start].parent_index)
    end
end

using Images
using Plots
image = load("images/field.png")
# make the image the background of the plot
anim = @animate for i in 1:length(x_hist)
    plot()
    plot!([0,domain_size*AR],[0,domain_size],reverse(image, dims = 1), yflip = false, aspect_ratio = :auto)
    plotFeatures2D(features_hist[i], :white)

    destination_indices_back = zeros(Int64, length(backup_hist[i]))
    for j in 1:length(backup_hist[i])
        destination_indices_back[j] = find_destination_index(backup_hist[i], j)
    end
    x_hist_x = [x[1] for x in x_hist[1:i]]
    x_hist_y = [x[2] for x in x_hist[1:i]]
    plot!(backup_hist[i], destination_indices_back, backup_planner.prob.turning_radius, colors = [:orange, [:white for i=1:length(backup_planner.prob.landmarks)-2]...,:yellow])
    success, best_path = query_nominal_planner(nominal_planner,  x_hist[i])
    if success
        best_path = make_path_from_waypoints(best_path, nominal_planner.prob.turning_radius)
        for p in best_path
            plot!(p, linewidth=5, color=:black, linestyle=:solid)
        end
    end

    # plot!(nominal_planner.nodes, destination_indices_nom, nominal_planner.prob.turning_radius, colors = [:gray, :cyan])
    plot!(x_hist_x, x_hist_y, color = :blue, linewidth = 5, linestyle = :solid)

    plotFOV(x_hist[i][1:2], x_hist[i][3], vo_params.fovRadius, vo_params.fovAngle)

    plot!(gk_hist[i]; linewidth=5)

    # goal
    plot!(Shape(backup_planner.prob.landmarks[end][1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
            backup_planner.prob.landmarks[end][2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
      color=:cyan, alpha=0.2)

    plot!([backup_planner.prob.landmarks[end][1]], [backup_planner.prob.landmarks[end][2]], color = :yellow, marker = :circle, markersize = 10)

    # start
    plot!(Shape(backup_planner.prob.landmarks[1][1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
    backup_planner.prob.landmarks[1][2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
    color=:orange, alpha=0.2)
    plot!([backup_planner.prob.landmarks[1][1]], [backup_planner.prob.landmarks[1][2]], color = :orange, marker = :circle, markersize = 10)
    
    for j in 2:length(backup_planner.prob.landmarks)-1
        plot!(Shape(backup_planner.prob.landmarks[j][1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
        backup_planner.prob.landmarks[j][2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
        color=:white, alpha=0.2)
        plot!([backup_planner.prob.landmarks[j][1]], [backup_planner.prob.landmarks[j][2]], color = :white, marker = :circle, markersize = 10)
    end

    plot_domain(nominal_planner.prob.domain)
    plot!(nominal_planner.prob.unsafe_zones; color=:red, width=2)
    plot!(legend=false, axis = false, grid=false, widen=false,background_color=:transparent, foreground_color=:black)
    xlims!(0, domain_size*AR)
    ylims!(0, domain_size)
    
    xlabel!("East [m]")
    ylabel!("North [m]")
end

In [ ]:
gif(anim, "field.gif", fps = 10)

In [ ]:
plot()

using Images
image = load("images/field.png")
plot!([0,domain_size*AR],[0,domain_size],reverse(image, dims = 1), yflip = false, aspect_ratio = :auto)
x_hist_x = [x[1] for x in x_hist]
x_hist_y = [x[2] for x in x_hist]
# push!(x_hist_x, goal[1])
# push!(x_hist_y, goal[2])
plotFeatures2D(features_hist[end], :white)
plot!(x_hist_x, x_hist_y, color = :blue, linewidth = 2)
for  l in backup_planner.prob.landmarks
      plot!([l[1]], [l[2]], color = :cyan, marker = :circle)
      plot!(Shape(l[1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
                  l[2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
            color=:cyan, alpha=0.2)
end

plot!([x_hist[1][1]], [x_hist[1][2]], color = :green, marker = :circle)
plot!(legend=false)
# plot!([mid_landmark[1]], [mid_landmark[2]], color = :yellow, marker = :circle, legend  = false)

In [ ]:
plot((1:length(cost_hist))*ΔT, cost_hist, xlabel="Time [s]", ylabel="Pos. Error [m]", legend=false)
plot!([(1, max_cost), (length(cost_hist)*ΔT, max_cost)], linestyle=:dash, color=:red)

In [ ]:
plot()
plot!((1:length(num_feats_hist))*ΔT, num_feats_hist, xlabel="Time [s]", ylabel="# Features Seen", legend=false)
plot!([(1, 8), (length(num_feats_hist)*ΔT, 8)], linestyle=:dash, color=:red)
x_start = NaN
for i in 2:length(num_feats_hist)
    if !isnan(num_feats_hist[i-1]) && isnan(num_feats_hist[i])
        x_start = (i-1) * ΔT
    elseif isnan(num_feats_hist[i-1]) && !isnan(num_feats_hist[i]) && !isnan(x_start)
        x_end = (i-1) * ΔT
        plot!(Shape([x_start, x_end, x_end, x_start], [0, 0, 80, 80]), color=:gray, alpha=0.3, legend=false)
        x_start = NaN
    end
end
ylims!(0, 80)

In [ ]:
dist_to_goal = [norm(x[SOneTo(2)] - backup_planner.prob.landmarks[end]) for x in x_hist]
plot((1:length(dist_to_goal))*ΔT, dist_to_goal, xlabel="Time", ylabel="Distance to goal", legend=false)

In [ ]:
Random.seed!(1337)
@time omniscient = 
    initialize_backup_planner(domain,
        vo_params,
        [goal],
        MVector(100.0,100.0,150.0),
        SizedVector(features),
        unsafe_zones,
        3000)

omniscient_cost, omniscient_path = query_backup_planner(omniscient, x0)
omniscient_path = make_path_from_waypoints(omniscient_path, omniscient.prob.turning_radius)
println("omniscient cost: ", omniscient_cost)

In [ ]:
nom_cost = compute_nominal_trajectory_cost(omniscient.prob, initial_nom_path, Inf)
nom_fail = length(nom_cost)
x_nom_fail = sample_combined_dubins_path(initial_nom_path, nom_fail*nominal_planner.prob.max_velocity/vo_params.updateRate)
println("nominal cost: ", nom_cost[end], " failed at ", x_nom_fail)

using Images
image = load("images/field.png")
plot([0,domain_size*AR],[0,domain_size],reverse(image, dims = 1), yflip = false, aspect_ratio = :auto)
xlims!(0, domain_size*AR)
ylims!(0, domain_size)
# plot!(omniscient.nodes, Int64[], omniscient.prob.turning_radius, colors = [:cyan])
# plot!(nominal_planner.nodes, Int64[], omniscient.prob.turning_radius, colors = [:cyan])
plotFeatures2D(features, :white)
plot!(Shape(backup_planner.prob.landmarks[end][1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
            backup_planner.prob.landmarks[end][2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
      color=:cyan, alpha=0.2)
# plot!(Shape(backup_planner.prob.landmarks[2][1] .+ backup_planner.prob.turning_radius*cos.(0:0.01:2pi),
#     backup_planner.prob.landmarks[2][2] .+ backup_planner.prob.turning_radius*sin.(0:0.01:2pi)), 
# color=:gray, alpha=0.2)
for p in omniscient_path
    plot!(p, linewidth=5, color=:magenta)
end
for p in initial_nom_path
    plot!(p, linewidth=5, color=:black)
    e, endpt = dubins_path_endpoint(p)
    if endpt[1] == x_nom_fail[1]+0
    # plotFOV(endpt[1:2], endpt[3], vo_params.fovRadius, vo_params.fovAngle)
    end
end
# plot!([x_nom_fail[1]], [x_nom_fail[2]], color = :red, marker = :excl, markersize = 10)
plotFOV(x_nom_fail[1:2], x_nom_fail[3], vo_params.fovRadius, vo_params.fovAngle)
plot!(unsafe_zones; color=:red, width=2)
plot!([x0[1]], [x0[2]], color = :orange, marker = :circle, markersize = 10)

plot!([backup_planner.prob.landmarks[end][1]], [backup_planner.prob.landmarks[end][2]], color = :yellow, marker = :circle, markersize = 10)

# plot!([backup_planner.prob.landmarks[2][1]], [backup_planner.prob.landmarks[2][2]], color = :gray, marker = :circle, markersize = 10)
plot!(legend=false, grid=false)

xlabel!("East [m]")
ylabel!("North [m]")